In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import io
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os.path

train_data = pd.read_csv("drive/My Drive/M5_traindata2/sales_train_validation.csv")
calendar_data = pd.read_csv("drive/My Drive/M5_traindata2/calendar.csv")
sellprices2_data = pd.read_csv("drive/My Drive/M5_traindata2/sell_prices2.csv")
calendar_data2 = pd.read_csv("drive/My Drive/M5_traindata2/calendar2.csv")

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap", "price", "price_down", "price_up", "value"]
train_data2a0 = pd.DataFrame(index=[], columns=cols)

start_n = 0

print(len(train_data))

for ii in range(start_n, len(train_data)):
    if (os.path.isfile("drive/My Drive/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False): #ファイルがまだ作成されていない
        train_data2a = train_data2a0.copy()
        tra_sir = train_data.iloc[ii]
        datas_base = tra_sir[0:6].tolist()

        sap_mindata = sellprices2_data[sellprices2_data.item_id == datas_base[1]]  # sellpricesの中から必要なデータを抽出
        sap_mindata = sap_mindata[sap_mindata.store_id == datas_base[4]]

        for jj in range(1913):
            cal_sir2 = calendar_data.iloc[jj]
            cal2_sir2 = calendar_data2.iloc[jj]
            sm2 = sap_mindata[sap_mindata.wm_yr_wk == cal_sir2[1]]  # sellpricesの該当行を抽出
            try:
                sap_sir = sm2.iloc[0]

                datas = copy.deepcopy(datas_base)
                kk = [cal_sir2[1], cal_sir2[3], cal_sir2[4], cal_sir2[5], cal_sir2[6], cal2_sir2[1], cal2_sir2[2], cal_sir2[3],
                      cal_sir2[4]]
                datas = datas + kk

                if (datas[5] == "CA"):
                    datas.append(cal2_sir2[5])  # 自分のいる州のsnapを調査
                elif (datas[5] == "TX"):
                    datas.append(cal2_sir2[6])
                elif (datas[5] == "WI"):
                    datas.append(cal2_sir2[7])
                else:
                    print(5 / 0)


                kk2 = [sap_sir[3], sap_sir[4], sap_sir[5], tra_sir[6 + jj]]
                datas = datas + kk2

                kari_t = pd.Series(datas, index=cols)
                train_data2a = train_data2a.append(kari_t, ignore_index=True)

            except:  # 該当価格データが存在しない=販売していない
                pass

        train_data2a.to_csv("drive/My Drive/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
    print('\r %d' % ii, end='')

30490


FileNotFoundError: ignored